In [1]:
import pysam
import numpy as np
import os
import math
from scipy import stats
import get_area_match_model as gamm
from functools import  partial
import get_training_data_0 as zero_area
import get_training_data_1 as one_area

In [2]:
ch = ['chr1','chr2','chr3','chr4','chr5',
      'chr6','chr7','chr8','chr9','chr10',
      'chr11','chr12','chr13','chr14','chr15',
      'chr16','chr17','chr19','chr20',
     'chr21']
vcf_file_path = r'/Volumes/big_ksg/lyx_del_code/del_vcf/'
#format in vcf_file_path：'chr18\t10050\t10659\t1\n'
depth_file_path = r'/Volumes/big_ksg/lyx_del_code/chr_depth/'
#format in depth_file_path：'chr18\tpos\tdepth\n'
bam_file_path = r'/Volumes/big_ksg/55x/'  #sorted_final_merged.bam
ml_save_path_zero = r'/Volumes/big_ksg/lyx_del_code/ml_training_inf/zero/'
ml_save_path_one = r'/Volumes/big_ksg/lyx_del_code/ml_training_inf/one/'
dl_save_path_zero = r'/Volumes/big_ksg/lyx_del_code/dl_pic/0/'
dl_save_path_one = r'/Volumes/big_ksg/lyx_del_code/dl_pic/1/'

In [8]:
def var_exp(x):
    return (math.exp((x/20)**0.5)-1)/math.exp((x/20)**0.5)

def binarySearch (arr, l, r, x): 
    if r >= l: 
        mid = int(l + (r - l)/2)
        if int(arr[mid].strip().split('\t')[1]) == x: 
            return mid 
        elif int(arr[mid].strip().split('\t')[1]) > x: 
            return binarySearch(arr, l, mid-1, x) 
        else: 
            return binarySearch(arr, mid+1, r, x)   
    else: 
        return -1

In [4]:
def div0(a,b):
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide(a,b)
        c[ ~ np.isfinite(c)] = 0
    return c

def get_single_area_pysam_inf(single_area_list,open_bam_file,chid):
    pysam_read_count_primary = 0
    pysam_read_count_secondary = 0
    for read in open_bam_file.fetch(contig=chid,start=single_area_list[0],stop=single_area_list[1]):
        if read.is_secondary == False:
            pysam_read_count_primary += 1
        elif read.is_secondary == True:
            pysam_read_count_secondary += 1
    return pysam_read_count_primary,pysam_read_count_secondary

In [10]:
def get_area_feature(final_single_areas,bam_file_path,save_path,chid,open_depth_file,open_depth_file_len,variation_or_not='',window_length=100,stride=75): #此步骤保存特征为txt文件，方便访问

    save_file = open(save_path+chid+'_get_'+variation_or_not+'_area_feature.txt',"w+")
    temp = np.zeros(shape=(1,18))
    open_bam_file = pysam.AlignmentFile(bam_file_path+'sorted_final_merged_'+chid+'.bam','rb')
    for i in range(len(final_single_areas)):
        temp_array_up,temp_array_now,temp_array_down = [],[],[] 
        depth_count_up,depth_count_now,depth_count_down = [],[],[]
        
        now_area = final_single_areas[i] 
        
        result_now = binarySearch(open_depth_file,0,open_depth_file_len,now_area[0])
        
        depth_count_now = [int(open_depth_file[result_now+j1].strip().split('\t')[2]) for j1 in range(window_length)]
        
        area_depth_average_now = np.mean(np.array(depth_count_now)) 
        
        area_depth_var_now = np.var(np.array(depth_count_now)) 
        
        count_del_percent_now,count_soft_clip_percent_now,count_primary_percent_now,read_num_now = gamm.match_model(bam_file_path,chid,now_area[0],now_area[1])
        
        now_feature = np.array([#now_area[0],now_area[1],
                                count_primary_percent_now
                                ,var_exp(area_depth_var_now)
                                ,read_num_now
                                ,count_del_percent_now
                                ,count_soft_clip_percent_now
                                ,area_depth_average_now])

        final_sample_features = now_feature
        final_sample_features = ['%.7s'%str(final_sample_feature) for final_sample_feature in final_sample_features]
        
        save_file.write(str(now_area[0])+'-'+str(now_area[1])+'\t'
                        +final_sample_features[0]+'\t' 
                        +final_sample_features[1]+'\t' 
                        +final_sample_features[2]+'\t' 
                        +final_sample_features[3]+'\t' 
                        +final_sample_features[4]+'\t' 
                        +final_sample_features[5]+'\t'
                        +str(1 if variation_or_not == 'one' else 0)+'\n')
    print(chid+' write_'+variation_or_not+'_sample_feature ... done')
    open_bam_file.close()
    #return final_sample_features